In [1]:
#Imporing the necessary libraries
import os
import pandas as pd
import numpy as np
import statistics as stat

### 1. Read the dataset, display the first few rows to understand it, and display the count of ratings (rows) in the dataset.

In [2]:
#Load the ratings data
baseLocation = "E:/OneDrive - TUNI.fi/Tampere University (MSc in CS-DS)/01. Autumn 2021/Period 2/1. Courses/2. Recommender Systems/Assignment_1/Dataset/ml-latest-small/"
ratings_data = pd.read_csv(baseLocation+"ratings.csv")

# Inspecting top 5 ratings data
ratings_data.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
# Drop 'timestamp' column from ratings data as it is not needed.
ratings_data.drop('timestamp', inplace=True, axis=1)

# Inspecting top 5 ratings data
ratings_data.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [4]:
# Count the rows
print(len(ratings_data))

# Number of users
print(len(pd.unique(ratings_data['userId'])))

100836
610


In [5]:
# Loading the movie data
movies_data = pd.read_csv(baseLocation+'movies.csv')

# Inspecting top 5 movies data
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# Drop 'genres' column from movies data as it is not needed.
movies_data.drop('genres', inplace=True, axis=1)

# Inspecting top 5 ratings data
movies_data.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


### 2. Implement the user-based collaborative filtering approach, using the Pearson correlation function for computing similarities between users

To implement a user based collaborative filtering approach, at first we select a user. 
For example, here we choose an user having a userId of 155. After that, we find out all the similar users like 155 using 
Pearson Correlation method. 


In [7]:
# Getting all the movies of a selected user has seen and rated
# This will work as an input values and these values will be used to recommend this user some new movies

# Selected userId (155) you can use any value
uId = 155
inputMovies = ratings_data[ratings_data['userId']== uId]
inputMovies.head(10)


,userId,movieId,rating
22638,155,1,3.0
22639,155,141,3.0
22640,155,153,3.0
22641,155,260,4.0
22642,155,333,4.0
22643,155,432,4.0
22644,155,575,3.0
22645,155,588,3.0
22646,155,592,3.0
22647,155,671,3.0


In [8]:
# Number of movies that selected user has rated/watched
len(inputMovies)

46

In [9]:
# Now we will find out the similar users like our selected user
# Alternately, we can say, finding the useres who rates similar movies as our selected user
similarUsers = ratings_data[ratings_data['movieId'].isin(inputMovies['movieId'].tolist())]
similarUsers

,userId,movieId,rating
0,1,1,4.0
15,1,260,5.0
18,1,333,5.0
33,1,592,4.0
92,1,1377,3.0
...,...,...,...
99709,610,3114,5.0
99736,610,3578,5.0
99753,610,3744,3.0
99754,610,3745,3.0


In [10]:
# Now creating sub dataframes based on userId
similarUsersGroup = similarUsers.groupby(['userId'])

In [11]:
# Inspecting one random user, let's say userId = 414
similarUsersGroup.get_group(414)

,userId,movieId,rating
62294,414,1,4.0
62352,414,141,4.0
62358,414,153,2.0
62410,414,260,5.0
62439,414,333,3.0
62484,414,432,2.0
62552,414,588,4.0
62555,414,592,4.0
62576,414,671,4.0
62725,414,1200,5.0


In [12]:
# Let's sort those users dataframes
# So that, users with most in common with the selected user will have the priority
similarUsersGroup = sorted(similarUsersGroup, key= lambda x:len(x[1]), reverse = True)

In [13]:
# Inspecting the 2nd top user (1st one is the same as our selected user)
similarUsersGroup[1]

(414,
        userId  movieId  rating
 62294     414        1     4.0
 62352     414      141     4.0
 62358     414      153     2.0
 62410     414      260     5.0
 62439     414      333     3.0
 62484     414      432     2.0
 62552     414      588     4.0
 62555     414      592     4.0
 62576     414      671     4.0
 62725     414     1200     5.0
 62784     414     1302     4.0
 62805     414     1377     4.0
 62842     414     1517     5.0
 62848     414     1562     2.0
 62851     414     1580     4.0
 62930     414     1883     3.0
 62939     414     1917     4.0
 62944     414     1923     5.0
 62957     414     1968     5.0
 62980     414     2028     5.0
 63097     414     2355     4.0
 63160     414     2502     4.0
 63188     414     2628     4.0
 63213     414     2724     1.0
 63276     414     2948     3.0
 63280     414     2959     5.0
 63307     414     3052     3.0
 63327     414     3114     5.0
 63342     414     3175     4.0
 63448     414     3555     3.0
 6

In [14]:
# Number of similar users, who rated all the movies or some of them.
len(similarUsersGroup)

558

In [15]:
# userId of the 2nd top user group
similarUsersGroup[1][0]

414

In [16]:
# Dataframe of the 2nd top user group
similarUsersGroup[1][1]

,userId,movieId,rating
62294,414,1,4.0
62352,414,141,4.0
62358,414,153,2.0
62410,414,260,5.0
62439,414,333,3.0
62484,414,432,2.0
62552,414,588,4.0
62555,414,592,4.0
62576,414,671,4.0
62725,414,1200,5.0


In [17]:
#let's check how many movies this user has rated
len(similarUsersGroup[1][1])

35

### Calculating The Similarity (Pearson Correlation) of the Selected (Input) User to the Similar User

In [18]:
# Number of similar users is very high (558) and not all the similar users are not contributing well for the recommendation
# Here, limiting the number to 100 Similar Users while computing the Similarity Score using pearson corrrelation.

similarUsersGroup = similarUsersGroup[1:101] # Removing the first user (userId =155), as it is our selected user
len(similarUsersGroup)

100

In [19]:
#Store the Pearson Correlation in a dictionary, 
# where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every similar user group in our subset
for name, group in similarUsersGroup:
    
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    
    #Get the N (total similar movies watched) for the formula 
    #nRatings = len(group)
    
    #Get the review scores of the Selected user for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    selectedUserTempRatingList = temp_df['rating'].tolist()
    #print(selectedUserTempRatingList)
    
    #Let's also put the current user group reviews in a list format
    similarUserTempRatingList = group['rating'].tolist()
    
    #Calculating pearson similarity
    simXX = 0
    meanSelectedUserTempRating = stat.mean(selectedUserTempRatingList)
    for i in selectedUserTempRatingList:
        simXX = simXX + pow((i - meanSelectedUserTempRating),2)
    
    simYY = 0
    meanSimilarUserTempRaning = stat.mean(similarUserTempRatingList)
    for j in similarUserTempRatingList:
        simYY = simYY + pow((j - meanSimilarUserTempRaning),2)
        
    simXY = 0
    for i, j in zip(selectedUserTempRatingList, similarUserTempRatingList):
        simXY = simXY+ ((i - meanSelectedUserTempRating ) * (j-meanSimilarUserTempRaning))
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if simXX != 0 and simYY != 0:
        pearsonCorrelationDict[name] = simXY/np.sqrt(simXX*simYY)
    else:
        pearsonCorrelationDict[name] = 0                    
    
    

In [20]:
pearsonCorrelationDict.items()

dict_items([(414, -0.030167041664563982), (599, 0.11234830900809734), (608, 0.4464873000525615), (182, -0.35700946822620605), (68, 0.19353991202063656), (274, -0.180794369528993), (19, 0.204019442118549), (448, -0.20065119361545156), (438, -0.005971453032500175), (288, 0.15233821244244802), (474, 0.18122313057747416), (480, 0.02663263505755903), (45, 0.09617635677666722), (91, 0.1640089328533902), (483, 0.16610262808732373), (64, -0.02666755560000242), (135, 0.027068534274164054), (177, -0.2386305222485107), (219, -0.02327361260929778), (294, 0.2123284749005513), (380, 0.09303609266337155), (590, -0.12781316401923645), (217, 0.18553546269482762), (226, 0.0033965502454676596), (249, 0.06552389669095604), (292, -0.1150777042837601), (307, -0.21272570488609935), (453, -0.051841896887813814), (555, -0.03853081288900599), (489, -0.3624022845584481), (600, 0.17194120423502943), (42, 0.13880363049242067), (82, 0.6155784217875951), (387, 0.2550121784479975), (573, 0.19595922445781594), (57, 0.

In [21]:
similarityScoreofSimilarUsers_data = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
similarityScoreofSimilarUsers_data.head()

,0
414,-0.030167
599,0.112348
608,0.446487
182,-0.357009
68,0.193540


In [22]:
#Similarity score of other similar users to our selected user (155)
similarityScoreofSimilarUsers_data.columns = ['similarityScore']
similarityScoreofSimilarUsers_data['userId'] = similarityScoreofSimilarUsers_data.index
similarityScoreofSimilarUsers_data.index = range(len(similarityScoreofSimilarUsers_data))
similarityScoreofSimilarUsers_data.head()

,similarityScore,userId
0,-0.030167,414
1,0.112348,599
2,0.446487,608
3,-0.357009,182
4,0.193540,68


### The top 10 similar users to input user

In [23]:
topSimilarUsers=similarityScoreofSimilarUsers_data.sort_values(by='similarityScore', ascending=False)
topSimilarUsers.head(10)

,similarityScore,userId
96,0.765478,93
81,0.625485,522
41,0.624204,580
32,0.615578,82
2,0.446487,608
93,0.403113,560
45,0.392232,115
56,0.331674,596
83,0.329350,122
84,0.312505,140


### Calculating Recommendation Score

In [24]:
# Merging the top similar users similarity score with their ratings of movies
topSimilarUsersRating = topSimilarUsers.merge(ratings_data, left_on='userId', right_on='userId', how='inner')
topSimilarUsersRating.head()

,similarityScore,userId,movieId,rating
0,0.765478,93,1,3.0
1,0.765478,93,2,5.0
2,0.765478,93,10,4.0
3,0.765478,93,15,4.0
4,0.765478,93,34,5.0


In [25]:
# Calculating the mean ratings of all the similar users
meanRb = topSimilarUsersRating.groupby('userId').mean()[['rating']]
meanRb.columns = ['avgRating']
meanRb['userId'] = meanRb.index
meanRb.index = range(len(meanRb))
meanRb

,avgRating,userId
0,4.366379,1
1,3.230263,7
2,2.607397,19
3,3.260722,21
4,3.020175,28
...,...,...
95,3.507953,603
96,3.657399,606
97,3.786096,607
98,3.134176,608


In [26]:
# Merging the top similar users average rating score 
topSimilarUsersRating = topSimilarUsersRating.merge(meanRb, left_on='userId', right_on='userId', how='inner')
topSimilarUsersRating.head()

,similarityScore,userId,movieId,rating,avgRating
0,0.765478,93,1,3.0,4.28866
1,0.765478,93,2,5.0,4.28866
2,0.765478,93,10,4.0,4.28866
3,0.765478,93,15,4.0,4.28866
4,0.765478,93,34,5.0,4.28866


In [27]:
#Multiplies the similarity by the user's ratings
topSimilarUsersRating['weightedRatingScore'] = topSimilarUsersRating['similarityScore']*(topSimilarUsersRating['rating']-topSimilarUsersRating['avgRating'])
topSimilarUsersRating.head()

,similarityScore,userId,movieId,rating,avgRating,weightedRatingScore
0,0.765478,93,1,3.0,4.28866,-0.986441
1,0.765478,93,2,5.0,4.28866,0.544515
2,0.765478,93,10,4.0,4.28866,-0.220963
3,0.765478,93,15,4.0,4.28866,-0.220963
4,0.765478,93,34,5.0,4.28866,0.544515


In [28]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopSimilarUsersRating = topSimilarUsersRating.groupby('movieId').sum()[['weightedRatingScore']]
tempTopSimilarUsersRating.columns = ['sum_weightedRatingScore']
tempTopSimilarUsersRating['movieId'] = tempTopSimilarUsersRating.index
tempTopSimilarUsersRating.head()

,sum_weightedRatingScore,movieId
movieId,,
1,-3.430038,1
2,0.409365,2
3,-1.812607,3
4,-0.256447,4
5,-0.492905,5


In [29]:
#Creates an empty dataframe
recommendation_data = pd.DataFrame()
#Now we take the weighted average
meanRa = inputMovies['rating'].mean()
recommendation_data['weighted average recommendation score'] = meanRa+(tempTopSimilarUsersRating['sum_weightedRatingScore']/topSimilarUsers['similarityScore'].sum())
recommendation_data['movieId'] = recommendation_data.index
recommendation_data.head()


,weighted average recommendation score,movieId
movieId,,
1,3.005972,1
2,3.680629,2
3,3.290186,3
4,3.563633,4
5,3.522083,5


### Recommended Movies 

In [30]:
recommendation_data = recommendation_data.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_data.head(20)

,weighted average recommendation score,movieId
movieId,,
2028,4.936516,2028
2628,4.925712,2628
1210,4.742338,1210
260,4.735162,260
3578,4.686289,3578
6934,4.625742,6934
2571,4.625165,2571
1196,4.613628,1196
5378,4.551882,5378


### Top 20 Recommended Movies

In [31]:
# Top 20 Recommended Movies
movies_data.loc[movies_data['movieId'].isin(recommendation_data.head(20)['movieId'].tolist())]

,movieId,title
224,260,Star Wars: Episode IV - A New Hope (1977)
257,296,Pulp Fiction (1994)
507,589,Terminator 2: Judgment Day (1991)
898,1196,Star Wars: Episode V - The Empire Strikes Back...
900,1198,Raiders of the Lost Ark (Indiana Jones and the...
902,1200,Aliens (1986)
911,1210,Star Wars: Episode VI - Return of the Jedi (1983)
990,1291,Indiana Jones and the Last Crusade (1989)
1183,1580,Men in Black (a.k.a. MIB) (1997)
1298,1732,"Big Lebowski, The (1998)"
